## Use the model end point locally 

we can use the model currently in the local and get the output as json return  

In [2]:
import requests
import json

# url to access the end point of the model
url = "http://localhost:11434/api/generate"

# pass the json object with the data to selecting model and the context
data = {
    "model": "gemma:2b",
    "prompt" : "Once upon a time"
}

response = requests.post(url, json=data, stream=True)

# check the response status
if response.status_code == 200:
    # iterate over the streaming response
    for line in response.iter_lines():
        if line:
            decode_line = line.decode("utf-8")
            result = json.loads(decode_line)

            # get the text from the response
            generated_txt = result.get("response","")
            print(generated_txt, end="", flush=True)
else:
    print("error code : ",response)


A little boy named Alex was sitting on a rock by the river. He was watching the boats go by, and he wondered what it would be like to travel on one of them. He had never been in a boat before, but he was sure it would be an amazing experience.

<center>Alex's eyes lit up with excitement as he imagined himself sailing down the river in a boat.</center><br>

**How would you like to help Alex explore his dream of sailing on a boat?**

## Use of the ollama library

In [3]:
import ollama

response =  ollama.list()
print(response)

models=[Model(model='qwen:4b', modified_at=datetime.datetime(2025, 2, 14, 12, 25, 45, 67058, tzinfo=TzInfo(+05:30)), digest='d53d04290064542e5f12ccfb0055785b7751264dc6bbb06c04c559c57e07496a', size=2330093361, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='4B', quantization_level='Q4_0')), Model(model='gemma:2b', modified_at=datetime.datetime(2025, 2, 14, 11, 42, 16, 672324, tzinfo=TzInfo(+05:30)), digest='b50d6c999e592ae4f79acae23b4feaefbdfceaa7cd366df2610e3072c052a160', size=1678456656, details=ModelDetails(parent_model='', format='gguf', family='gemma', families=['gemma'], parameter_size='3B', quantization_level='Q4_0')), Model(model='nomic-embed-text:latest', modified_at=datetime.datetime(2025, 2, 14, 10, 30, 30, 764904, tzinfo=TzInfo(+05:30)), digest='0a109f422b47e3a30ba2b10eca18548e944e8a23073ee3f3e947efcf3c45e59f', size=274302450, details=ModelDetails(parent_model='', format='gguf', family='nomic-bert', families=['nomic-be

In [4]:
# get the simple directory reader from llamaIndex
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# import pdf files with the llamaIndex SimpleDirectoryReader
# eBook-How-to-Build-a-Career-in-AI.pdf
documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]),"\n")
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'> 

Doc ID: 209cbac0-34b8-4eb9-bdac-296dd8d24950
Text: PAGE 1 Founder, DeepLearning.AI Collected Insights from Andrew
Ng How to  Build Your Career in AI A Simple Guide


In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext

In [7]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings


llm = Ollama(model="qwen:4b")

response = llm.complete("Who is Laurie Voss? write in 10 words")
print(response)

Laurie Voss is a marketing executive.


In [8]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="qwen:4b", request_timeout=60.0)

response = llm.complete("What is the capital of France?")
print(response)

Paris.


In [9]:
from llama_index.core.schema import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))
print(document)

Doc ID: 341b709a-20b6-4973-af9a-cf596db562ee
Text: PAGE 1 Founder, DeepLearning.AI Collected Insights from Andrew
Ng How to  Build Your Career in AI A Simple Guide  PAGE 2 "AI is the
new  electricity. It will  transform and improve  all areas of human
life." Andrew Ng  PAGE 3 Table of  Contents Introduction: Coding AI is
the New Literacy. Chapter 1: Three Steps to Career Growth. Chapter 2:
Learn...


In [10]:
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model="qwen:4b", temperature=0.1,request_timeout=120)
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# use for global configure the models and embedding models
# Settings.llm = llm
# Settings.embed_model=embed_model

index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)


In [11]:
query_engine = index.as_query_engine(llm=llm)

In [12]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)

print(str(response))

When finding projects to build your experience, here are some steps you can take:

1. Identify areas of interest: Start by identifying areas of interest that align with your career goals. This could include topics such as natural language processing, computer vision, or machine learning.

2. Research existing projects: Once you have identified areas of interest, research existing projects in those areas. This will give you an idea of the types of projects that are currently being worked on in those areas.

3. Network with domain experts: Once you have researched existing projects and networked with domain experts, you will have a better understanding of the types of projects that are currently being worked on in those areas, as well as the skills and experience required to successfully build and execute such projects.
4. Apply for internships or entry-level positions: Once you have a good understanding of the types of projects that are currently being worked on in those areas, as well 

## Run with deep seek model

In [13]:
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1,request_timeout=120)
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# use for global configure the models and embedding models
# Settings.llm = llm
# Settings.embed_model=embed_model
# index the documents based on the given embedding model
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

In [14]:
# make query engine to query the given question and retrieve relevant document
# and pass to the llm to get the solution based on given context
query_engine = index.as_query_engine(llm=llm)

In [15]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

<think>
Okay, so I need to figure out how to find project ideas when there aren't any available. The user provided some context about career development in AI, specifically about building a portfolio of projects that show skill progression. They mentioned starting small and gradually moving to bigger projects as skills grow.

First, the context talks about page 18, which is about identifying and scoping AI projects. It says we should focus on responsible, ethical, and beneficial projects. But it doesn't mention anything about finding new ideas or project ideas. So maybe I need to look elsewhere in the context for guidance on how to find project ideas.

Looking at the context again, there's a section about page 22, which is about career development. It mentions starting small with homework assignments and then moving to more complex projects as skills grow. But it doesn't directly address finding new ideas or project ideas.

I remember that in the previous chapter, the author wrote abou

In [16]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [18]:
! nvidia-smi

Wed Feb 26 09:23:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   58C    P8              3W /   40W |    2447MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
import requests
import json

# url to access the end point of the model
url = "http://localhost:11434/api/generate"

# pass the json object with the data to selecting model and the context
data = {
    "model": "deepseek-r1:1.5b",
    "prompt" : "Once upon a time"
}

response = requests.post(url, json=data, stream=True)

# check the response status
if response.status_code == 200:
    # iterate over the streaming response
    for line in response.iter_lines():
        if line:
            decode_line = line.decode("utf-8")
            result = json.loads(decode_line)

            # get the text from the response
            generated_txt = result.get("response","")
            print(generated_txt, end="", flush=True)
else:
    print("error code : ",response)


<think>

</think>

Once upon a time, in a small, ancient kingdom named by its people rather than a name that evokes any notion of modernity or grandeur, there lived a community of animals and humans who lived together harmoniously. This was not the best of times, as the kingdom had been on the brink of collapse due to a series of natural disasters, disease outbreaks, and societal ills.

One day, an old, wise man named Ophelia decided to step in and save her people. She had come from afar to protect their offspring from the impending disaster. As she led them to safety, she met numerous creatures who greeted her with a mix of curiosity and confusion. One such creature was a group of animals that lived on the edge of extinction—a rare species known as "The Viperones."

As Ophelia explained the situation to the Veptines, they realized that their true purpose was not just survival but also a way of life. Their members were wise creatures who had been selected by Ophelia for their intellige

## find the relevant loaded model and find the loaded model using CPU/GPU
* run the command "ollama ps" in the cmd to find the running model using CPU/GPU
* OLLAMA_USE_CUDA=1 to force ollama to use GPU instead of using CPU

In [22]:
OLLAMA_USE_CUDA=1
! nvidia-smi

Sun Feb 23 09:42:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   48C    P8              2W /   40W |    1523MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Set Ollama as Evaluation model in the Deep-eval
* deepeval set-ollama deepseek-r1:1.5b

To unset the model we can use 
* deepeval unset-ollama

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

llm = Ollama(model="qwen:4b", temperature=0.1,request_timeout=120)
# Replace this with the actual output from your LLM application
actual_output = "We offer a 30-day full refund at no extra cost."

metric = AnswerRelevancyMetric(
    # score required for an answer to be considered relevant
    threshold=0.7,
    # model being used to evaluate the relevance
    model="qwen:4b",
    # function will also return an explanation or reason for the relevance score
    include_reason=True
)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0
The answer is relevant because it directly addresses the question about shoe fitting and provides a helpful response.


In [31]:
import pprint
pprint.pprint(test_cases)

[LLMTestCase(input='What are the keys to building a career in AI?',
             actual_output=Response(response='<think>\n'
                                             'Okay, so I need to figure out '
                                             'what the key points are for '
                                             'building a career in AI based on '
                                             'the provided context. Let me '
                                             'start by reading through the '
                                             'context carefully.\n'
                                             '\n'
                                             'First, it mentions that AI is '
                                             'still evolving and not as mature '
                                             'as other fields. That probably '
                                             'means foundational skills like '
                                             'machi

In [34]:
# change the responses 
for test in test_cases:
    test.actual_output = test.actual_output.response

In [39]:
# Run evaluation
evaluate(test_cases=[test_cases[0]], metrics=[answer_relevancy], identifier="First Run")

✨ You're running DeepEval's latest Answer Relevancy Metric! (using local model, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:41, 41.92s/test case]




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: local model, reason: The answer relevancy score is 1.00 because all relevant statements addressed the query about building a career in AI, and there are no irrelevant statements to affect this conclusion., error: None)

For test case:

  - input: What are the keys to building a career in AI?
  - actual output: <think>
Okay, so I need to figure out what the key points are for building a career in AI based on the provided context. Let me start by reading through the context carefully.

First, it mentions that AI is still evolving and not as mature as other fields. That probably means foundational skills like machine learning and deep learning are crucial because they form the base of any AI project. Without these, you might struggle to understand more advanced topics later on.

Next, the context talks about facing challenges in project management, especially with AI projects that are h

✓ Tests finished 🎉! View results on 
]8;id=370108;https://app.confident-ai.com/project/cm7exqe4a0bt413kajni6zycg/evaluation/test-runs/cm7llf7ig0019vxr7nlrdmi56/test-cases\https://app.confident-ai.com/project/cm7exqe4a0bt413kajni6zycg/evaluation/test-runs/cm7llf7ig0019vxr7nlrdmi56/test-]8;;\
]8;id=370108;https://app.confident-ai.com/project/cm7exqe4a0bt413kajni6zycg/evaluation/test-runs/cm7llf7ig0019vxr7nlrdmi56/test-cases\cases]8;;\.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The answer relevancy score is 1.00 because all relevant statements addressed the query about building a career in AI, and there are no irrelevant statements to affect this conclusion.', strict_mode=False, evaluation_model='local model', error=None, evaluation_cost=0.0, verbose_logs='Statements:\n[\n    "AI is still evolving and not as mature as other fields.",\n    "Foundational technical skills like machine learning and deep learning are crucial.",\n    "Faced challenges in project management with AI projects requiring iterative improvements.",\n    "Worked on meaningful projects to apply technical skills effectively.",\n    "Built a supportive community of friends and allies for collaboration and guidance.",\n    "Understood the importance of job search processes in the competitive AI field.",\n    "Adapted conti

In [ ]:
from deepeval.test_case import LLMTestCase

# The input is the direct input to your prompt template, and so SHOULD NOT CONTAIN your prompt template
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    expected_output="You're eligible for a 30 day refund at no extra cost.",
    actual_output="We offer a 30-day full refund at no extra cost.",
    context=["All customers are eligible for a 30 day full refund at no extra cost."],
    retrieval_context=["Only shoes can be refunded."],
)